<a href="https://colab.research.google.com/github/Tossmax/Big_Cities_Health_Data_Inventory/blob/master/Copy_of_Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
pd.set_option('display.max_columns', None)

In [ ]:
train_df = pd.read_csv('/content/drive/My Drive/Train.csv')
test_df = pd.read_csv('/content/drive/My Drive/Train.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

In [ ]:
numerical_features = [col for col in train_df.columns if train_df[col].dtypes != object]
categorical_features = [col for col in train_df.columns if train_df[col].dtypes == object]

In [ ]:
numerical_cols = [col for col in test_df.columns if test_df[col].dtypes != object]
categorical_cols = [col for col in test_df.columns if test_df[col].dtypes == object]

In [ ]:
train_df[numerical_features] = train_df[numerical_features].fillna(-999)

In [ ]:
test_df[numerical_cols] = test_df[numerical_cols].fillna(-999)

In [ ]:
encoder = LabelEncoder()

tran_f = ['default_status', 'form_field47']

for col in tran_f:
    train_df[col] = encoder.fit_transform(train_df[col])

In [ ]:
label_encoder = LabelEncoder()

test_df['form_field47'] = label_encoder.fit_transform(test_df['form_field47'])

In [ ]:
feature_names = ['form_field1', 'form_field2',  'form_field3', 'form_field4', 'form_field5', 'form_field6', 'form_field7', 'form_field8', 'form_field9', 'form_field10', 'form_field11', 'form_field12', 'form_field13',  'form_field14', 'form_field15', 'form_field16', 'form_field17', 'form_field18', 'form_field20', 'form_field21', 'form_field22', 'form_field23', 'form_field24', 'form_field25', 'form_field26', 'form_field27', 'form_field28', 'form_field29', 'form_field30', 'form_field31', 'form_field32', 'form_field33', 'form_field34', 'form_field35', 'form_field36', 'form_field37', 'form_field38', 'form_field39', 'form_field40', 'form_field41', 'form_field42', 'form_field43', 'form_field44', 'form_field45', 'form_field46', 'form_field47', 'form_field48', 'form_field49', 'form_field50']
X = train_df[feature_names]
y = train_df.default_status

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier
from xgboost import XGBClassifier 
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier

In [ ]:
cat = CatBoostClassifier(n_estimators=100, thread_count=-1, od_type='Iter', loss_function='Logloss', verbose=False)
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
xg = XGBClassifier( n_jobs =-1)
lg = LGBMClassifier( n_jobs=-1)
lr = LogisticRegression()
minmax = MinMaxScaler()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [ ]:
use_classifier = [('cat', cat), ('xg', xg), ('lg', lg), ('rf', rf)]
model = VotingClassifier(estimators= use_classifier, voting='soft')
pipe = Pipeline(steps =[('scaler', minmax), ('model', model)])
pipe.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('model',
                 VotingClassifier(estimators=[('cat',
                                               <catboost.core.CatBoostClassifier object at 0x7fc537b51898>),
                                              ('xg',
                                               XGBClassifier(base_score=0.5,
                                                             booster='gbtree',
                                                             colsample_bylevel=1,
                                                             colsample_bynode=1,
                                                             colsample_bytree=1,
                                                             gamma=0,
                                                             learning_rate=0.1,
                                                             max_delta_step=0,
                                

In [ ]:
pipe.score(x_test, y_test)

0.8127142857142857

In [ ]:
feature_nam = ['form_field1', 'form_field2',  'form_field3', 'form_field4', 'form_field5', 'form_field6', 'form_field7', 'form_field8', 'form_field9', 'form_field10', 'form_field11', 'form_field12', 'form_field13',  'form_field14', 'form_field15', 'form_field16', 'form_field17', 'form_field18', 'form_field20', 'form_field21', 'form_field22', 'form_field23', 'form_field24', 'form_field25', 'form_field26', 'form_field27', 'form_field28', 'form_field29', 'form_field30', 'form_field31', 'form_field32', 'form_field33', 'form_field34', 'form_field35', 'form_field36', 'form_field37', 'form_field38', 'form_field39', 'form_field40', 'form_field41', 'form_field42', 'form_field43', 'form_field44', 'form_field45', 'form_field46', 'form_field47', 'form_field48', 'form_field49', 'form_field50']
X_test = test_df[feature_nam]

In [ ]:
from sklearn.metrics import accuracy_score
score = roc_auc_score(y_test, pipe.predict_proba(x_test)[:, 1])
print(score)

0.8413107893881696


In [ ]:
from skopt.space import Real, Categorical, Integer

In [ ]:
splits = 35

skfold = StratifiedKFold(n_splits=splits, random_state=42, shuffle=True)
kfold = KFold(n_splits=splits, shuffle=True, random_state=42)

In [ ]:
scores = []
preds = []

vc = VotingClassifier(estimators=[('cat', CatBoostClassifier(verbose=False, random_state=22, thread_count= -1)), 
                                 ('lg', LGBMClassifier(n_jobs= -1, objective= 'Binary', metric='AUC',  random_state=32)), ('rf', RandomForestClassifier(n_estimators=500, n_jobs=-1))], voting = 'soft')
i = 1

for train_split, test_split in skfold.split(X, y):
    pipe = Pipeline([('scaler', minmax), ('model', vc)])
    x_train, x_test, y_train, y_test = X.iloc[train_split], X.iloc[test_split], y.iloc[train_split], y.iloc[test_split]
    pipe.fit(x_train, y_train)
    print('number of splits trained {} '.format(i))
    score = roc_auc_score(y_test, pipe.predict_proba(x_test)[:, 1])
    pred = pipe.predict_proba(X_test)[:, 1]
    scores.append(score)
    preds.append(pred)
    i += 1
np.mean(score)

number of splits trained 1 
number of splits trained 2 
number of splits trained 3 
number of splits trained 4 
number of splits trained 5 
number of splits trained 6 
number of splits trained 7 
number of splits trained 8 
number of splits trained 9 
number of splits trained 10 
number of splits trained 11 
number of splits trained 12 
number of splits trained 13 
number of splits trained 14 
number of splits trained 15 
number of splits trained 16 
number of splits trained 17 
number of splits trained 18 
number of splits trained 19 
number of splits trained 20 
number of splits trained 21 
number of splits trained 22 
number of splits trained 23 
number of splits trained 24 
number of splits trained 25 
number of splits trained 26 
number of splits trained 27 
number of splits trained 28 
number of splits trained 29 
number of splits trained 30 


In [ ]:
TOO_submission = pd.DataFrame({"Applicant_ID": test_df["Applicant_ID"], "default_status" :  pred})

In [ ]:
TOO_submission.to_csv('Predica.csv', index=False)